In [1]:
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_breast_cancer
from sklearn import model_selection

import warnings

from sklearn.linear_model import LogisticRegression as LR

In [2]:
X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.25)

### Адаптивный бустинг на деревьях решений

In [3]:
def get_error(pred, y):
    return np.sum((pred != y).astype(int)) / len(y)

In [4]:
def adaboost(X, y, N):

    n_objects = len(X)

    n_classes = len(np.unique((y)))

    w = np.ones(n_objects) / n_objects

    models = []

    for n in range(N):
        clf = DecisionTreeClassifier(max_depth=1)
        clf.fit(X, y, w)

        predictions = clf.predict(X)
        e = get_error(predictions, y)
        if e >= 1 - 1/n_classes: 
            break

        alpha = 0.5 * np.log((1 - e) / e)

        match = predictions == y

        w[np.logical_not(match)] *= np.exp(alpha)
        w[match] *= np.exp(-alpha)

        w /= w.sum()

        models.append((alpha, clf))
    
    return models

In [5]:
def predict(X, models):
    
    n_classes = 2
    n_objects = len(X)
    
    y_pred = np.zeros((n_objects, n_classes))
    
    for alpha, clf in models:
        prediction = clf.predict(X)
        y_pred[range(n_objects), prediction] += alpha
    
    y_pred = np.argmax(y_pred, axis=1)
    
    return y_pred

In [6]:
N = 50

models = adaboost(X_train, y_train, N)

In [7]:
print(f'Точность алгоритма на обучающей выборке: {(1 - get_error(predict(X_train, models), y_train)) * 100:.3f}')

Точность алгоритма на обучающей выборке: 95.070


In [8]:
print(f'Точность алгоритма на тестовой выборке: {(1 - get_error(predict(X_test, models), y_test)) * 100:.3f}')

Точность алгоритма на тестовой выборке: 90.909


### Адаптивный бустинг, использующий Логистическую Регрессию

In [9]:
def log_loss(pred, y):
    return -(np.sum(y*np.log2(pred)+(1-y)*np.log2(1-pred))/len(y))

In [10]:
def adaboost_lr(X, y, N):

    n_objects = len(X)

    n_classes = len(np.unique((y)))

    w = np.ones(n_objects) / n_objects

    models = []

    for n in range(N):
        clf = LR()
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            clf.fit(X, y)

        predictions = clf.predict_proba(X)[:, 1]
        e = log_loss(predictions, y)
        if e >= 1 - 1/n_classes: 
            break

        alpha = 0.5 * np.log((1 - e) / e)

        match = clf.predict(X) == y

        w[np.logical_not(match)] *= np.exp(alpha)
        w[match] *= np.exp(-alpha)

        w /= w.sum()

        models.append((alpha, clf))
    
    return models

In [11]:
N = 50

models = adaboost_lr(X_train, y_train, N)

In [12]:
clf = LR()
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    clf.fit(X_train, y_train)
    predictions = clf.predict_proba(X_train)[:, 1]

In [13]:
print(f'Точность алгоритма на обучающей выборке: {(1 - log_loss(predictions, y_train)) * 100:.3f}')

Точность алгоритма на обучающей выборке: 83.554


In [14]:
clf = LR()
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    clf.fit(X_test, y_test)
    predictions = clf.predict_proba(X_test)[:, 1]

In [15]:
print(f'Точность алгоритма на тестовой выборке: {(1 - log_loss(predictions, y_test)) * 100:.3f}')

Точность алгоритма на тестовой выборке: 82.167


### Сравнение результатов

Точность адаптивного бустинга на деревьях решений выше, чем точность адаптивного бустинга, использующего Логистическую Регрессию